In [13]:
import os
import telebot
import speech_recognition
from pydub import AudioSegment
# Загрузка изображения для демо
import urllib
from PIL import Image, ImageEnhance, ImageFilter
from IPython.display import display


# Ниже нужно вставить токен, который дал BotFather при регистрации
# Пример: token = '2007628239:AAEF4ZVqLiRKG7j49EC4vaRwXjJ6DN6xng8'
token = '7904252722:AAGmfTau2g6eFqCUBI0Yxr6Hmqgj2gcibSE'  # <<< Ваш токен

bot = telebot.TeleBot(token, threaded=False)


def transform_image(filename):
    # Функция обработки изображения
    source_image = Image.open(filename)
    enhanced_image = source_image.filter(ImageFilter.EMBOSS)
    enhanced_image = enhanced_image.convert('RGB')
    enhanced_image.save(filename)
    return filename


@bot.message_handler(content_types=['photo'])
def resend_photo(message):
    # Функция отправки обработанного изображения
    file_id = message.photo[-1].file_id
    filename = download_file(bot, file_id)

    # Трансформируем изображение
    transform_image(filename)

    image = open(filename, 'rb')
    bot.send_photo(message.chat.id, image)
    image.close()

    # Не забываем удалять ненужные изображения
    if os.path.exists(filename):
        os.remove(filename)


def oga2wav(filename):
    # Конвертация формата файлов
    new_filename = filename.replace('.oga', '.wav')
    audio = AudioSegment.from_file(filename)
    audio.export(new_filename, format='wav')
    return new_filename

def download_file(bot, file_id):
    # Скачивание файла, который прислал пользователь
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id + file_info.file_path
    filename = filename.replace('/', '_')
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    return filename

def recognize_speech(oga_filename):
    # Перевод голоса в текст + удаление использованных файлов
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()

    with speech_recognition.WavFile(wav_filename) as source:
        wav_audio = recognizer.record(source)

    text = recognizer.recognize_google(wav_audio, language='ru')

    if os.path.exists(oga_filename):
        os.remove(oga_filename)

    if os.path.exists(wav_filename):
        os.remove(wav_filename)

    if not os.path.exists(oga_filename):
       bot.send_message("Файл voice.ogg не найден.")

    return text


@bot.message_handler(commands=['start'])
def say_hi(message):
    # Функция, отправляющая "Привет" в ответ на команду /start
    bot.send_message(message.chat.id, 'Привет,' + message.chat.first_name + '\n' +
                     'Я - Первый практически применимый результат подготовки Фёдора Соловьёва для развития в програмировании.' + '\n' +
                     'Я умею распознавать слова в аудиосообщениях и выводить их пользователю на экран. Теперь содержание голосового от друга можно узнать прямо на паре!')


@bot.message_handler(content_types=['voice'])
def transcript(message):
    # Функция, отправляющая текст в ответ на голосовое
    filename = download_file(bot, message.voice.file_id)
    text = recognize_speech(filename)
    bot.send_message(message.chat.id, text)


# Запускаем бота. Он будет работать до тех пор, пока работает ячейка
# (крутится значок слева).
# Остановим ячейку - остановится бот
bot.polling()

FileNotFoundError: [WinError 2] Не удается найти указанный файл

In [2]:
import os
import telebot
from pydub import AudioSegment
import speech_recognition as sr

API_TOKEN = '7904252722:AAGmfTau2g6eFqCUBI0Yxr6Hmqgj2gcibSE'
bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(content_types=['voice'])
def handle_voice(message):
    file_info = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    with open('voice.oga', 'wb') as new_file:
        new_file.write(downloaded_file)

    # Конвертация OGG в WAV
    audio = AudioSegment.from_ogg('voice.oga')
    audio.export('voice.wav', format='wav')

    # Распознавание речи
    recognizer = sr.Recognizer()
    with sr.AudioFile('voice.wav') as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='ru-RU')
            bot.reply_to(message, text)
        except sr.UnknownValueError:
            bot.reply_to(message, "Не удалось распознать голос.")
        except sr.RequestError:
            bot.reply_to(message, "Ошибка сервиса распознавания.")

if __name__ == '__main__':
    bot.polling(none_stop=True)

C:\Users\fshun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] Не удается найти указанный файл